# Sentence Generator

Generating sentences.

For most of these demos, rerun the cell to get a new sentence output

In [18]:
from random import choice, random
from ipywidgets import Output, Button

def coin():
    """
    Return a random boolean
    """
    return random() > 0.5

Start with simplest sentence: a noun (the subject) a verb and another noun (the object).

In [63]:
noun_a = [
    'John',
    'Jim',
    'Gordon Ramsay',
    'Steven',
    'Khan',
    'Roy',
    'Michael',
    'Vladmir Putin'
]

verb = [
    'jumps',
    'runs',
    'cooks',
    'annoys',
    'helps',
    'reads',
    'chooses'
]

noun_b = [
    'the fence',
    'the shark',
    'the child',
    'the mistress',
    'the working class',
    'the book'
]

choice(noun_a) + ' ' + choice(verb) + ' ' + choice(noun_b)

'Roy annoys the working class'

Kinda redundant... We have two sets of nouns, could we combine them?

Well, nouns come in two flavors, proper and improper. Improper nouns have articles ("a/an" and "the"), while proper nouns don't (kind of). So we can have a set for proper nouns, and a set for improper nouns, and pick between them randomly

In [78]:
nouns_proper = [
    'John',
    'Jim',
    'Gordon Ramsay',
    'Steven',
    'Khan',
    'Roy',
    'Michael',
    'Vladmir Putin'
]

nouns_improper = [
    'fence',
    'shark',
    'child',
    'mistress',
    'citizen',
    'book',
    'egg'
]

verbs = [
    'jumps',
    'runs',
    'cooks',
    'annoys',
    'helps',
    'reads',
    'chooses',
    'writes',
    'punches'
]

def noun():
    if coin():
        return choice(nouns_proper)
    else:
        return 'the ' + choice(nouns_improper)

# Following the apparent convention
def verb():
    return choice(verbs)
    
# Geneate sentence
noun() + ' ' + verb() + ' ' + noun()

'the fence helps the citizen'

Now we got a bit more intrigue... the improper nouns only begin with "the", since there's a bit more code that needs to go into "a/an". Namely, we have to programatically choose between "a" and "an" depending on if the noun starts with a vowel.

Also, I'm going to imply that any noun that doesn't allow for both "a/an" and "the" is proper (e.g. "the working class")

In [123]:
nouns_proper = [
    'John',
    'Jim',
    'Gordon Ramsay',
    'Steven',
    'Khan',
    'Roy',
    'Michael',
    'Vladmir Putin',
    'the working class',
    'the biker gang',
    'the sky',
    'France'
]

nouns_improper = [
    'fence',
    'shark',
    'child',
    'mistress',
    'citizen',
    'book',
    'egg'
]

verbs = [
    'jumps',
    'runs',
    'cooks',
    'annoys',
    'helps',
    'reads',
    'chooses',
    'writes',
    'punches'
]

def noun():
    if coin():
        return choice(nouns_proper)
    else:
        noun = choice(nouns_improper)
        if coin():
            return 'the ' + noun
        elif noun[0] in 'aeiou': # Usually not y
            return 'an ' + noun
        else:
            return 'a ' + noun
            

# Following the apparent convention
def verb():
    return choice(verbs)
    
# Generate sentence
noun() + ' ' + verb() + ' ' + noun()

'Gordon Ramsay punches a fence'

Now comes the first tough part: plural nouns. You can have one citizen cook an egg, or multiple citizens cook an egg. However, if we want to implement this, it will introduce new problems. We need a singular and plural form of a noun. With most nouns you just attach an 's' to the end, but _some_ nouns are special (e.g. person singular, people plural). So we have to account for both. _Also_, the verb will change depending on wether the noun is singular or plural. So now we introduce dependencies.

We'll have to create a staged approach. We first determine our subject. Then, from there, we format the verb based on the subject's plurality, and finally we introduce the object.

As for singular and plural, special nouns will be provided in arrays, and plain strings imply add an 's'.

In [166]:
nouns_improper = [
    'fence',
    'shark',
    ['child', 'children'],
    ['mistress', 'misstresses'],
    'citizen',
    'book',
    'egg'
]

nouns_proper = [
    'John',
    'Jim',
    'Gordon Ramsay',
    'Steven',
    'Khan',
    'Roy',
    'Michael',
    'Vladmir Putin',
    'the working class',
    'the biker gang',
    'the sky',
    'France'
]

verbs = [
    'jump',
    'run',
    'cook',
    'annoy',
    'help',
    'read',
    'choose',
    'write',
    ['punch', 'punches']
]

def noun_improper(plural):
    value = choice(nouns_improper)
    if plural:
        if type(value) is list:
            word = value[1]
        else:
            word = value + 's'
        return 'the ' + word
    else:
        if type(value) is list:
            word = value[0]
        else:
            word = value
        if coin():
            if word[0] in 'aeiou': # usually not y
                return 'an ' + word
            else:
                return 'a ' + word
        else:
            return 'the ' + word
    
def noun_proper():
    return choice(nouns_proper)

def noun():
    if coin():
        plural = coin()
        return noun_improper(plural), plural
    else:
        return noun_proper(), False
    
def verb(plural):
    value = choice(verbs)
    if type(value) is list and plural:
        return value[0]
    elif type(value) is list:
        return value[1]
    elif plural:
        return value
    else:
        return value + 's'
    
# Build sentence
subject, plural = noun()
action = verb(plural)
object_, _ = noun()
subject + ' ' + action + ' ' + object_

'the working class cooks Khan'

We're starting to make sentences more interesting. However, the code is kind of getting unruly, so I'm going to do some refactoring

In [241]:
class RandomValue:
    @classmethod
    def random(cls):
        value = choice(cls.data)
        return cls(*value) if type(value) is list else cls(value)
    

class ProperNoun(RandomValue):
    data = [
        'John',
        'Jim',
        'Gordon Ramsay',
        'Steven',
        'Khan',
        'Roy',
        'Michael',
        'Vladmir Putin',
        'the working class',
        'the biker gang',
        'the sky',
        'France'
    ]
    
    def __init__(self, word):
        self.word = word
        
    def get_word(self, plural):
        return self.word

    
class ImproperNoun(RandomValue):
    data = [
        'fence',
        'shark',
        ['child', 'children'],
        ['mistress', 'misstresses'],
        'citizen',
        'book',
        'egg'
    ]
    
    def __init__(self, singular, plural=None):
        self.singular = singular
        self.plural = plural or (singular + 's')
        
    def get_word(self, plural):
        word = self.plural if plural else self.singular
        article = 'the' if plural or coin() else 'an' if word[0] in 'aeiou' else 'a'
        return article + ' ' + word


def noun():
    if coin():
        value = ImproperNoun.random()
        plural = coin()
        return value.get_word(plural), plural
    else:
        return ProperNoun.random().get_word(False), False
    
    
class Verb(RandomValue):
    data = [
        'jump',
        'run',
        'cook',
        'annoy',
        'help',
        'read',
        'choose',
        'write',
        ['punch', 'punches']
    ]
        
    def __init__(self, plural, singular=None):
        self.plural = plural
        self.singular = singular or (plural + 's')
        
    def get_word(self, plural):
        return self.plural if plural else self.singular
    

# Build sentence
subject, plural = noun()
action = Verb.random().get_word(plural)
object_, _ = noun()
subject + ' ' + action + ' ' + object_

'Jim writes a book'